# Unterabfragen

- Mehrere Abfragen werden verschachtelt oder gejoint.
    
- Die Unterabfrage wird in runde Klammern gesetzt.
    
- Um eine Unterabfrage zu joinen, bekommt sie ein Alias wie eine Tabelle.
    
- Unterabfragen können unabhängig sein  oder abhängig von der umgebenden Abfrage.
    
- Abhängige Unterabfragen nennt man **korrelierte Unterabfragen**.
    
- Unterabfragen kommen in verschiedenen Klauseln vor:
    
    - SELECT Klausel
    - FROM Klausel
    - WHERE Klausel
    - HAVING Klausel

Es folgen zunächst Beispiele mit Unterabfragen in verschiedenen Klauseln, danach korrelierte Unterabfragen:

## Unterabfragen in der SELECT-Klausel

In [2]:
SELECT COUNT(*) FROM Kunde

(1 row affected)

Total execution time: 00:00:00.044

(No column name)
92


In [ ]:
-- Unabhängige Unterabfrage in der SELECT Klausel
-- Typisch zur Berechnung von Prozentanteilen für Gruppen
SELECT Land 
	, COUNT(*) AS [Anzahl Kunden]
	, 100.0 * COUNT(*) / (
		SELECT COUNT(*) FROM Kunde -- Unterabfrage!
	) AS [Prozent unformatiert]
	, CAST(100.0 * COUNT(*) / (
		SELECT COUNT(*) FROM Kunde -- Unterabfrage!
	) AS decimal(4,1)) AS [Anteil in %]
FROM Kunde
GROUP BY Land
ORDER BY [Anzahl Kunden] DESC;

In [ ]:
SELECT SUM(Menge * Verkaufspreis) FROM BestellDetail

In [ ]:
-- Zeigen Sie in SQL_Seminar: Gesamtumsatz nach Land gruppiert und Umsatzanteil in Prozent
SELECT k.Land,
    SUM(d.Menge * d.Verkaufspreis) AS Umsatz,
    CAST(100.0 * SUM(d.Menge * d.Verkaufspreis) / (
        -- Unterabfrage für Gesamtumsatz aller Länder
        SELECT SUM(Menge * Verkaufspreis) FROM BestellDetail
    ) AS decimal(5,2)) AS [Umsatzanteil %]
FROM Kunde AS k
JOIN Bestellung AS b ON k.KundenNr = b.KundenNr
JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
GROUP BY k.Land
ORDER BY k.Land ASC;

-- Zeigen Sie in SQL_Seminar: Gesamtumsatz nach Produktkategorie gruppiert und Umsatzanteil in Prozent

-- Zeigen Sie in SQL_Seminar für das Jahr 2017: Gesamtumsatz nach Produktkategorie gruppiert und Umsatzanteil in Prozent 

In [ ]:
-- Zeigen Sie in SQL_Seminar: Gesamtumsatz nach Produktkategorie gruppiert und Umsatzanteil in Prozent
SELECT k.Kategorie,
    SUM(d.Menge * d.Verkaufspreis) AS Umsatz,
    CAST(100.0 * SUM(d.Menge * d.Verkaufspreis) / (
        -- Unterabfrage für Gesamtumsatz aller Länder
        SELECT SUM(Menge * Verkaufspreis) FROM BestellDetail
    ) AS decimal(5,2)) AS [Umsatzanteil %]
FROM Produktkategorie AS k
JOIN Produkt AS p ON k.KategorieID = p.KategorieID
JOIN BestellDetail AS d ON p.ProduktID = d.ProduktID
GROUP BY k.Kategorie
ORDER BY k.Kategorie ASC;

In [ ]:
-- Zeigen Sie in SQL_Seminar für das Jahr 2017: Gesamtumsatz nach Produktkategorie gruppiert und Umsatzanteil in Prozent
SELECT k.Kategorie,
    SUM(d.Menge * d.Verkaufspreis) AS Umsatz,
    CAST(100.0 * SUM(d.Menge * d.Verkaufspreis) / (
        -- Unterabfrage für Gesamtumsatz aller Länder
        SELECT SUM(Menge * Verkaufspreis) 
        FROM BestellDetail AS d
        JOIN Bestellung AS b ON b.BestellungID = d.BestellungID
        WHERE YEAR(b.Bestelldatum) = 2017
    ) AS decimal(5,2)) AS [Umsatzanteil %]
FROM Produktkategorie AS k
JOIN Produkt AS p ON k.KategorieID = p.KategorieID
JOIN BestellDetail AS d ON p.ProduktID = d.ProduktID
JOIN Bestellung AS b ON b.BestellungID = d.BestellungID
WHERE YEAR(b.Bestelldatum) = 2017
GROUP BY k.Kategorie
ORDER BY k.Kategorie ASC;

In [ ]:
-- Wenn die Unterabfrage immer denselben Wert liefert,
-- lässt der sich vorab in eine Variable speichern
DECLARE @anzahlKunden INT = (
	SELECT COUNT(*) FROM Kunde
	);
SELECT Land
	, COUNT(*) AS [Anzahl Kunden]
	, CAST((100.0 * COUNT(*) / @anzahlKunden) AS decimal(4,1)) AS [Anteil in %]
FROM Kunde
GROUP BY Land
ORDER BY [Anzahl Kunden] DESC;

In [ ]:
SELECT BestellungID
		, SUM(Menge * Verkaufspreis) AS Rechnungsbetrag
	FROM BestellDetail
	GROUP BY BestellungID 

## Unterabfragen in der FROM-Klausel

In [ ]:
-- Unabhängige Unterabfrage in der FROM Klausel
-- z. B. Details bereitstellen, aus denen Aggregate gebildet werden
SELECT FORMAT(MAX(Rechnungsbetrag),'C','de-de') AS [Größte Rechnungssumme]
	, FORMAT(MIN(Rechnungsbetrag),'C','de-de') AS [Kleinste Rechnungssumme]
	, FORMAT(AVG(Rechnungsbetrag),'C','de-de') AS [Durchschnittliche Rechnungssumme]
	, FORMAT(SUM(Rechnungsbetrag),'C','de-de') AS [Gesamtumsatz]
FROM
( 
	SELECT BestellungID
		, SUM(Menge * Verkaufspreis) AS Rechnungsbetrag
	FROM BestellDetail
	GROUP BY BestellungID 
) AS Umsatzdaten;

In [ ]:
-- Alternativ: Unterabfrage mit WITH voranstellen
WITH Umsatzdaten AS (
	SELECT BestellungID
		, SUM(Menge * Verkaufspreis) AS Rechnungsbetrag
	FROM BestellDetail
	GROUP BY BestellungID
)
SELECT FORMAT(MAX(Rechnungsbetrag),'C','de-de') AS [Größte Rechnungssumme]
	, FORMAT(MIN(Rechnungsbetrag),'C','de-de') AS [Kleinste Rechnungssumme]
	, FORMAT(AVG(Rechnungsbetrag),'C','de-de') AS [Durchschnittliche Rechnungssumme]
	, FORMAT(SUM(Rechnungsbetrag),'C','de-de') AS [Gesamtumsatz]
FROM Umsatzdaten;

In [ ]:
    SELECT DISTINCT KundenNr
    FROM Bestellung
    WHERE Bestelldatum >= '20170101'
    AND Bestelldatum < '20170201' 

In [ ]:
-- WHERE Klausel
-- z. B. ausgewählte IDs liefern
-- Zeige Firma und Ort der Kunden, die im Januar 2017 bestellt haben
SELECT Firma, Ort
FROM Kunde
WHERE KundenNr IN ( 
    SELECT DISTINCT KundenNr
    FROM Bestellung
    WHERE Bestelldatum >= '20170101'
    AND Bestelldatum < '20170201' 
);

## Unterabfragen in der WHERE-Klausel

In [ ]:
-- Das geht auch ohne Unterabfrage!
SELECT k.Firma, k.Ort
FROM Kunde AS k
JOIN Bestellung AS b 
	ON k.KundenNr = b.KundenNr
WHERE Bestelldatum >= '20170101'
AND Bestelldatum < '20170201';

In [ ]:
-- Übungen: Unterabfragen in der WHERE-Klausel:
-- Bitte jeweils eine zweite Lösungsvariante ohne Unterabfrage entwickeln!
-- Welcher Mitarbeiter hat im Februar 2020 etwas verkauft? (1 Zeile)
SELECT Vorname, Nachname
FROM Mitarbeiter
WHERE MitarbeiterNr IN (
    SELECT MitarbeiterNr
    FROM Bestellung
    WHERE Bestelldatum >= '20200201'
    AND Bestelldatum < '20200301'
);
-- alternativ:
SELECT m.Vorname, m.Nachname
FROM Mitarbeiter AS m
JOIN Bestellung AS b ON m.MitarbeiterNr = b.MitarbeiterNr
WHERE b.Bestelldatum >= '20200201'
AND b.Bestelldatum < '20200301';

In [ ]:
-- Welche Produkte wurden im Februar 2020 verkauft? (2 Zeilen)
SELECT Produkt
FROM Produkt
WHERE ProduktID IN (
    SELECT ProduktID
    FROM Bestellung AS b 
    JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
    WHERE b.Bestelldatum >= '20200201'
        AND b.Bestelldatum < '20200301'
);

SELECT Produkt
FROM Produkt
WHERE ProduktID IN (
    SELECT ProduktID
    FROM BestellDetail
    WHERE BestellungID IN (
        SELECT BestellungID
        FROM Bestellung WHERE Bestelldatum <='20200228'
        AND Bestelldatum >'20200131'
    )
);

SELECT p.Produkt
FROM Produkt AS p
JOIN BestellDetail AS d ON p.ProduktID = d.ProduktID
JOIN Bestellung AS b ON b.BestellungID = d.BestellungID
WHERE Bestelldatum >= '20200201'
AND Bestelldatum < '20200301';

In [ ]:
SELECT TOP 1 SUM(d.Menge * d.Verkaufspreis) AS Wert
	FROM Bestellung AS b
	JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
	WHERE b.Bestelldatum >= '2017-01-01'
		AND b.Bestelldatum < '2018-01-01'
	GROUP BY b.BestellungID
	ORDER BY Wert DESC

In [ ]:
SELECT b.BestellungID, b.Bestelldatum
	, SUM(d.Menge * d.Verkaufspreis) AS Bestellwert
FROM Bestellung AS b
JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
WHERE b.Bestelldatum >= '2018-01-01'
	AND b.Bestelldatum < '2019-01-01'
GROUP BY  b.BestellungID, b.Bestelldatum

## Unterabfragen in der HAVING-Klausel

In [ ]:
-- Unterabfrage in HAVING Klausel
-- Filtern auf dem Wert aus einer Aggregation
-- Zeige Bestellungen aus 2018, die über dem höchsten Bestellwert aus 2017 liegen
-- BestellungID, Bestelldatum, Bestellwert / 11493.20
SELECT b.BestellungID, b.Bestelldatum
	, SUM(d.Menge * d.Verkaufspreis) AS Bestellwert
FROM Bestellung AS b
JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
WHERE b.Bestelldatum >= '2018-01-01'
	AND b.Bestelldatum < '2019-01-01'
GROUP BY  b.BestellungID, b.Bestelldatum
HAVING SUM(d.Menge * d.Verkaufspreis) > (
	-- höchster Bestellwert aus 2017
	SELECT TOP 1 SUM(d.Menge * d.Verkaufspreis) AS Wert
	FROM Bestellung AS b
	JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
	WHERE b.Bestelldatum >= '2017-01-01'
		AND b.Bestelldatum < '2018-01-01'
	GROUP BY b.BestellungID
	ORDER BY Wert DESC
);

In [ ]:
WITH Bestellungen2017 AS (
    SELECT SUM(d.Menge * d.Verkaufspreis) AS Wert
	FROM Bestellung AS b
	JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
	WHERE b.Bestelldatum >= '2017-01-01'
		AND b.Bestelldatum < '2018-01-01'
	GROUP BY b.BestellungID
)
SELECT b.BestellungID, b.Bestelldatum
	, SUM(d.Menge * d.Verkaufspreis) AS Bestellwert
FROM Bestellung AS b
JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
WHERE b.Bestelldatum >= '2018-01-01'
	AND b.Bestelldatum < '2019-01-01'
GROUP BY  b.BestellungID, b.Bestelldatum
HAVING SUM(d.Menge * d.Verkaufspreis) > (
	SELECT MAX(Wert) FROM Bestellungen2017
);

# Korrelierte Unterabfragen

- die Unterabfrage ist abhängig von der äußeren Abfrage
- sie kann deshalb nicht alleine ausgeführt werden
- Alternative zu LEFT JOIN
    - z. B.: "Zeige **ALLE** Kunden und die Anzahl ihrer Bestellungen"

In [ ]:
SELECT COUNT(BestellungID) 
		FROM Bestellung
        -- Korrelation!
		-- WHERE Bestellung.KundenNr = Kunde.KundenNr

In [ ]:
-- "Zeige ALLE Kunden und die Anzahl ihrer Bestellungen"
-- Lösung mit korrelierter Unterabfrage
SELECT Firma
	, (
		SELECT COUNT(BestellungID) 
		FROM Bestellung
        -- Korrelation!
		WHERE Bestellung.KundenNr = Kunde.KundenNr
	) AS [Anzahl Bestellungen]
FROM Kunde;

In [ ]:
-- "Zeige ALLE Kunden und die Anzahl ihrer Bestellungen"
-- Lösung mit LEFT JOIN
SELECT k.Firma
	, COUNT(b.BestellungID) AS [Anzahl Bestellungen]
FROM Kunde AS k
LEFT JOIN Bestellung AS b
	ON k.KundenNr = b.KundenNr
GROUP BY k.firma
ORDER BY [Anzahl Bestellungen] ASC;